In [ ]:
# sat xarray to np
import xarray as xr
from ocf_blosc2 import Blosc2
import numpy as np
from datetime import datetime
import os
from tqdm import tqdm
def process(year, month):
    import xarray as xr
    from ocf_blosc2 import Blosc2
    import numpy as np
    from datetime import datetime
    import os
    from tqdm import tqdm
    data = xr.open_dataset(
        f"/data/satellite-nonhrv/{year}/{month}.zarr.zip", engine="zarr", chunks="auto"
    )
    days = np.unique(data["time"].dt.floor("D").to_numpy())  # get unique days

    for day in tqdm(days):
        dt = to_datetime(day)
        folder_name = base_folder + dt.strftime('%y-%m-%d') + "/"
        file_name = "data.npz"
        total_name = folder_name + file_name

        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        times_of_day = data["time"].where(data["time"].dt.floor("D") == day, drop=True).to_numpy()
        data_of_day = {to_datetime(time).strftime('%H:%M:%S'): data["data"].sel(time=time).to_numpy() for time in times_of_day}

        np.savez(total_name, **data_of_day)  # save
def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)

base_folder = "/data/sat_np/"
from joblib import Parallel, delayed
for year in range(2021, 2022):
    Parallel(n_jobs=6)(delayed(process)(year, month) for month in range(1, 13))

In [ ]:
# weather xarray to np
import xarray as xr
from ocf_blosc2 import Blosc2
import numpy as np
from datetime import datetime
import os
from tqdm import tqdm

base_folder = "/data/weather_np/"

def process(year, month):
    import xarray as xr
    from ocf_blosc2 import Blosc2
    import numpy as np
    from datetime import datetime
    import os
    from tqdm import tqdm
    data = xr.open_dataset(
        f"/data/weather/{year}/{month}.zarr.zip", engine="zarr", chunks="auto"
    )
    NWP_FEATURES = [
    "t_500", "clcl", "alb_rad", "tot_prec", "ww",
    "relhum_2m", "h_snow", "aswdir_s", "td_2m", "omega_1000"
    ]
    data = data[NWP_FEATURES]
    days = np.unique(data["time"].dt.floor("D").to_numpy())  # get unique days

    for day in tqdm(days):
        dt = to_datetime(day)
        folder_name = base_folder + dt.strftime('%y-%m-%d') + "/"
        file_name = "data.npz"
        total_name = folder_name + file_name

        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        times_of_day = data["time"].where(data["time"].dt.floor("D") == day, drop=True).to_numpy()
        data_of_day = {}
        for time in times_of_day:
            to_stack = []
            for var in NWP_FEATURES:
                to_stack.append(data[var].sel(time=time).to_numpy())
            stacked = np.stack(to_stack)
            data_of_day[to_datetime(time).strftime('%H:%M:%S')] = stacked
        np.savez(total_name, **data_of_day)  # save
def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)

from joblib import Parallel, delayed
for year in range(2021, 2022):
    Parallel(n_jobs=6)(delayed(process)(year, month) for month in range(1, 13))